In [1]:
!pip install lightning -qqq
!pip install pytorch-lightning -qqq
!pip install pytorch-forecasting -qqq
!pip install optuna statsmodels -qqq
!pip install optuna-integration[pytorch_lightning] -qqq
!pip install tensorboard tensorboardX -qqq
!pip install darts[torch] -qqq

ERROR: Failed to build installable wheels for some pyproject.toml based projects (statsforecast)


### Lectura de archivos y preparación para entrenar

In [2]:
import os
from tqdm import tqdm
import torch, itertools, random
import pandas as pd
import numpy as np
from datetime import datetime
import urllib.request
import warnings
from pathlib import Path

import darts
from darts import TimeSeries
from darts.models import TFTModel
from darts.metrics import mae
from pytorch_lightning.callbacks import EarlyStopping
from darts.utils.losses import MAELoss
from sklearn.preprocessing import StandardScaler
from darts.dataprocessing.transformers import Scaler
from darts.utils.likelihood_models import QuantileRegression

import pickle
import glob
import multiprocessing

The StatsForecast module could not be imported. To enable support for the AutoARIMA, AutoETS and Croston models, please consider installing it.


In [3]:
# DEFINIR NUMERO DE EXPERIMENTO
experimento = 'darts_tfts'

In [4]:
# Configuración de rutas
#DRIVE_PATH = "/content/drive/MyDrive/tft_experiment"
#OPTUNA_PATH = f"{DRIVE_PATH}/optuna_results"
#MODEL_PATH = f"{DRIVE_PATH}/final_model"

# Crear directorios
#os.makedirs(OPTUNA_PATH, exist_ok=True)
#os.makedirs(MODEL_PATH, exist_ok=True)

In [44]:
df_tft_corto = pd.read_parquet('data/tft_ventana_corta.parquet')
df_tft_largo = pd.read_parquet('data/tft_completo.parquet')
df_tft_agg = pd.read_parquet('data/tft_agg.parquet')

baseline = pd.read_parquet('data/baseline.parquet')

In [45]:
df_tft_corto.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,timestamp,month
0,201701,10234,20524,0,2,0.05300,0.05300,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,2017-01-01,1
1,201701,10032,20524,0,1,0.13628,0.13628,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,2017-01-01,1
2,201701,10217,20524,0,1,0.03028,0.03028,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,2017-01-01,1
3,201701,10125,20524,0,1,0.02271,0.02271,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,2017-01-01,1
4,201701,10012,20524,0,11,1.54452,1.54452,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,2017-01-01,1


In [46]:
df_tft_largo.head()

,customer_id,product_id,timestamp,tn,cat1,cat2,cat3,brand,is_synth
0,10234,20524,2017-01-01,0.05300,HC,VAJILLA,Cristalino,Importado,False
1,10234,20524,2017-02-01,0.00000,None,None,None,None,True
2,10234,20524,2017-03-01,0.01514,HC,VAJILLA,Cristalino,Importado,False
3,10234,20524,2017-04-01,0.00000,None,None,None,None,True
4,10234,20524,2017-05-01,0.00000,None,None,None,None,True


In [47]:
df_tft_agg.head()

,product_id,timestamp,tn,cat1,cat2,cat3,brand,sku_size,descripcion,plan_precios_cuidados,periodo,month,is_synth
0,20001,2017-01-01,934.772217,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,0.0,201701.0,1.0,False
1,20001,2017-02-01,798.016174,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,0.0,201702.0,2.0,False
2,20001,2017-03-01,1303.357666,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,0.0,201703.0,3.0,False
3,20001,2017-04-01,1069.961304,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,0.0,201704.0,4.0,False
4,20001,2017-05-01,1502.201294,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,0.0,201705.0,5.0,False


In [48]:
baseline.head()

,product_id,tn
0,20001,1162.705930
1,20002,1183.639540
2,20003,684.763159
3,20004,580.484232
4,20005,563.559924


In [49]:
# agrego variables económicas
df_macro = pd.read_csv('data/datos_contexto_pais.csv')
df_macro["timestamp"] = pd.to_datetime(df_macro["periodo"], format="%Y%m").dt.to_period("M").dt.to_timestamp()
df_macro_clean = df_macro.drop(columns=["periodo"])		
df_macro_clean.head()

,IPC,inflacion,cambio_dolar,dias_feriados,timestamp
0,100,0.000000,15.890,2,2016-12-01
1,102,2.000000,16.080,1,2017-01-01
2,104,1.960784,15.800,2,2017-02-01
3,107,2.884615,15.645,1,2017-03-01
4,109,1.869159,15.490,1,2017-04-01


In [50]:
# extiendo variables macro

fechas_extra = pd.date_range("2020-01-01", "2020-02-01", freq="MS")
ultimo_macro = df_macro_clean[df_macro_clean["timestamp"] == df_macro_clean["timestamp"].max()].copy()
macro_extra = pd.concat([ultimo_macro]*len(fechas_extra), ignore_index=True)
macro_extra["timestamp"] = fechas_extra

df_macro_ext = pd.concat([df_macro_clean, macro_extra], ignore_index=True)
df_macro_ext = df_macro_ext.sort_values("timestamp").reset_index(drop=True)

In [51]:
num_cols = ['IPC', 'inflacion', 'cambio_dolar', 'dias_feriados']
for c in num_cols:
    df_macro_ext[c] = pd.to_numeric(df_macro_ext[c], errors='coerce').astype('float32')

In [52]:
df_tft_corto = df_tft_corto.merge(df_macro_ext, on="timestamp", how="left")
df_tft_largo = df_tft_largo.merge(df_macro_ext, on="timestamp", how="left")
df_tft_agg = df_tft_agg.merge(df_macro_ext, on="timestamp", how="left")

### Creamos series_id

In [53]:
df_tft_corto.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,timestamp,month,IPC,inflacion,cambio_dolar,dias_feriados
0,201701,10234,20524,0,2,0.05300,0.05300,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,2017-01-01,1,102.0,2.0,16.08,1.0
1,201701,10032,20524,0,1,0.13628,0.13628,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,2017-01-01,1,102.0,2.0,16.08,1.0
2,201701,10217,20524,0,1,0.03028,0.03028,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,2017-01-01,1,102.0,2.0,16.08,1.0
3,201701,10125,20524,0,1,0.02271,0.02271,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,2017-01-01,1,102.0,2.0,16.08,1.0
4,201701,10012,20524,0,11,1.54452,1.54452,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,2017-01-01,1,102.0,2.0,16.08,1.0


In [54]:
# Crear series_id para Darts
for df in (df_tft_corto, df_tft_largo):
    df["series_id"] = df["customer_id"].astype(str) + "_" + df["product_id"].astype(str)
    print(f"Series únicas en {len(df)}-fila df:", df["series_id"].nunique())

df_tft_agg["series_id"] = df_tft_agg["product_id"].astype(str)
print("Series únicas en df_tft_agg:", df_tft_agg["series_id"].nunique())

Series únicas en 2293481-fila df: 262805
Series únicas en 9460980-fila df: 262805
Series únicas en df_tft_agg: 780


In [55]:
# Mapping directo desde series_id a product_id usando el df de series cortas
series_to_product = (
    df_tft_corto[["series_id", "product_id"]]
      .drop_duplicates()
      .set_index("series_id")["product_id"]
      .to_dict()
)

print(f"Mapping creado: {len(series_to_product)} series")

Mapping creado: 262805 series


### Entrenamiento

In [59]:
tft_largo= {
    "series_list": None,
    "covariates_list": None,
    "train_series": None,
    "val_series": None,
    "scalers": None,     
    "model": None,
    "best_params": None,
    "metrics": {},
    "preds_val": None,
    "preds_feb20": None,
}

In [60]:
tft_agg = {
    "series_list": None,
    "covariates_list": None,
    "train_series": None,
    "val_series": None,
    "scalers": None,     
    "model": None,
    "best_params": None,
    "metrics": {},
    "preds_val": None,
    "preds_feb20": None,
}

In [61]:
tft_corto = {
    "series_list": None,
    "covariates_list": None,
    "train_series": None,
    "val_series": None,
    "scalers": None,     
    "model": None,
    "best_params": None,
    "metrics": {},
    "preds_val": None,
    "preds_feb20": None,
}

In [62]:
# Completo índices de tft_corto
# 1) Índices base
all_series = df_tft_corto["series_id"].unique()
full_range  = pd.date_range(
    df_tft_corto["timestamp"].min(),
    df_tft_corto["timestamp"].max(),
    freq="MS"
)

# 2) MultiIndex destino: producto cartesiano
idx = pd.MultiIndex.from_product(
    [all_series, full_range],
    names=["series_id", "timestamp"]
)

# 3) Reindexar de una sola vez
df_tft_corto = (
    df_tft_corto
        .set_index(["series_id", "timestamp"])
        .reindex(idx)                 # ← crea filas faltantes en bloque
        .reset_index()
)

In [64]:
# 4) Máscara + fillna
df_tft_corto["tn"] = df_tft_corto["tn"].fillna(0)
df_tft_corto["obs_mask"] = df_tft_corto["tn"].ne(0).astype("float32")

# 5) Covariables macro: propaga hacia delante en vectorizado
cov_cols = ["IPC", "inflacion", "cambio_dolar", "dias_feriados"]
df_tft_corto.sort_values("timestamp", inplace=True)
df_tft_corto[cov_cols] = df_tft_corto.groupby("series_id")[cov_cols].ffill()

In [66]:
# covariables dinámicas
cov_cols_corto = ["IPC", "inflacion", "cambio_dolar", "dias_feriados", "obs_mask"]
cov_cols_largo = ["IPC", "inflacion", "cambio_dolar", "dias_feriados", "is_synth"]

In [67]:
num_cols_corto = cov_cols_corto + ["tn"]
num_cols_largo = cov_cols_largo + ["tn"]

df_tft_corto[num_cols_corto] = df_tft_corto[num_cols_corto].astype("float32")
df_tft_largo[num_cols_largo] = df_tft_largo[num_cols_largo].astype("float32")

In [68]:
def make_series_and_covs_from_series_id(
    df: pd.DataFrame,
    cov_cols: list[str],
    time_col: str = "timestamp",
    value_col: str = "tn",
    freq: str = "MS",
) -> tuple[list[TimeSeries], list[TimeSeries]]:
    grouped = df.groupby("series_id")
    print(f"📊 {len(grouped)} series agrupadas")

    series_list, covariates_list = [], []
    print("🚀 Procesando grupos...")
    for series_id, group in tqdm(grouped, desc="Creando TimeSeries"):
        ts = TimeSeries.from_dataframe(
            group,
            time_col=time_col,
            value_cols=value_col,
            freq=freq
        )

        covs = TimeSeries.from_dataframe(
            group,
            time_col=time_col,
            value_cols=cov_cols,
            freq=freq
        )

        series_list.append(ts)
        covariates_list.append(covs)

    print(f"✅ {len(series_list)} series creadas")
    return series_list, covariates_list

In [24]:
tft_corto["series_list"], tft_corto["covariates_list"] = make_series_and_covs_from_series_id(
    df=df_tft_corto,
    cov_cols=cov_cols_corto,
    time_col="timestamp",
    freq="MS"
)

📊 262805 series agrupadas
🚀 Procesando grupos...


Creando TimeSeries: 100%|██████████| 262805/262805 [16:50<00:00, 260.03it/s]


✅ 262805 series creadas


In [69]:
tft_largo["series_list"], tft_largo["covariates_list"] = make_series_and_covs_from_series_id(
    df=df_tft_largo,
    cov_cols=cov_cols_largo,
    time_col="timestamp",
    freq="MS"
)

📊 262805 series agrupadas
🚀 Procesando grupos...


Creando TimeSeries: 100%|██████████| 262805/262805 [17:15<00:00, 253.68it/s]


✅ 262805 series creadas


### Validación tft_corto con 2018

In [73]:
# se intentó correr en colab, el val_loss vienen en nan y se detiene tras primera epcoh
# se deja código igualmente

def split_ts(ts, cut_off):
    return ts.split_before(cut_off)

cut_off = pd.Timestamp("2018-12-01")
train_list_18, val_list_18, cov_train_18, cov_val_18 = [], [], [], []

for ts, cov in zip(tft_corto["series_list"], tft_corto["covariates_list"]):
    tr, va   = split_ts(ts,  cut_off)
    ctr, cva = split_ts(cov, cut_off)
    train_list_18.append(tr)
    val_list_18.append(va)
    cov_train_18.append(ctr)
    cov_val_18.append(cva)

TypeError: 'NoneType' object is not iterable

In [27]:
# Concatenamos covariables de train + val
cov_full_18 = [
    ctr.append(cva)          # 5 meses + 1 mes  (jul‑dic 2018)
    for ctr, cva in zip(cov_train_18, cov_val_18)
]

In [32]:
# utils para búsqueda de hp

search_space = {
    "hidden_size"         : [64, 128],
    "n_heads"             : [4, 8],
    "dropout"             : [0.1, 0.2],
    "batch_size"          : [64, 128],
    "optimizer_kwargs"    : [{"lr": 1e-3}, {"lr": 5e-4}],
}

# --------- muestreador aleatorio reproducible --------
def sample_params(space, n_samples):
    keys, values = zip(*space.items())
    combos = list(itertools.product(*values))
    random.seed(42); random.shuffle(combos)
    for c in combos[:n_samples]:
        yield dict(zip(keys, c))

# --------- encoders y creador de modelos -------------
def build_encoders():
    return {
        # mes → seno / coseno
        "cyclic": {"past": ["month"]},

        # escala TODAS las covariables numéricas
        # (IPC, inflación, dólar, feriados, obs_mask, …)
        "transformer": Scaler(StandardScaler())
    }

In [ ]:
def create_tft_model_val(params, in_len, out_len):
    pl_trainer_kwargs = dict(
        accelerator = "gpu" if torch.cuda.is_available() else "cpu",
        devices     = 1     if torch.cuda.is_available() else None,
        callbacks   = [EarlyStopping(monitor="val_loss",
                                     patience=20, mode="min")],
        enable_progress_bar = True,
    )
    return TFTModel(
        input_chunk_length = in_len,
        output_chunk_length= out_len,
        hidden_size        = params["hidden_size"],
        num_attention_heads= params["n_heads"],
        dropout            = params["dropout"],
        batch_size         = params["batch_size"],
        n_epochs           = 150,
        lstm_layers        = 2,
        likelihood         = QuantileRegression([0.1, 0.5, 0.9]),
        use_reversible_instance_norm = True,
        optimizer_kwargs   = params["optimizer_kwargs"],
        add_encoders       = build_encoders(),
        add_relative_index = True,
        pl_trainer_kwargs  = pl_trainer_kwargs,
        random_state       = 42,
        save_checkpoints   = False,
    )

In [ ]:
N_TRIALS = 25
best_mae, best_params, best_model = np.inf, None, None

trial_iter = tqdm(sample_params(search_space, N_TRIALS),
                  total=N_TRIALS,
                  desc="Trials")

for i, hp in enumerate(trial_iter, 1):
    trial_iter.set_postfix(hp=hp)

    model = create_tft_model_val(hp, in_len=5, out_len=1)
    model.fit(series=train_list_18,
              past_covariates=cov_train_18,
              verbose=True)
     
    preds = model.predict(n=1, series=train_list_18, past_covariates=cov_full_18)
    score = mae(val_list_18, preds, reduction=np.mean)
    print(f"   MAE dic‑18: {score:.4f}")

    if score < best_mae:
        best_mae, best_params, best_model = score, hp, model
        print("   🏆 Nuevo mejor!")

print("\n🏁 Mejor MAE:", round(best_mae, 4))
print("Hiperparámetros óptimos:", best_params)

🔍 Trial 1/25: {'hidden_size': 128, 'n_heads': 8, 'dropout': 0.1, 'batch_size': 128, 'optimizer_kwargs': {'lr': 0.001}}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

### TFT_corto final

In [113]:
best_params = {
    "hidden_size":        64,
    "n_heads":            4,
    "dropout":            0.1,
    "batch_size":         64,
    "optimizer_kwargs":   {"lr": 1e-3},
}

In [141]:
def create_tft_model(params, in_len, out_len):
    pl_trainer_kwargs = dict(
        accelerator = "gpu" if torch.cuda.is_available() else "cpu",
        devices     = 1     if torch.cuda.is_available() else None,
        #callbacks   = [EarlyStopping(monitor="val_loss",patience=20, mode="min")],
        enable_progress_bar = True,
    )
    return TFTModel(
        input_chunk_length = in_len,
        output_chunk_length= out_len,
        hidden_size        = params["hidden_size"],
        num_attention_heads= params["n_heads"],
        dropout            = params["dropout"],
        batch_size         = params["batch_size"],
        n_epochs           = 60,
        lstm_layers        = 2,
        likelihood         = QuantileRegression([0.1, 0.5, 0.9]),
        use_reversible_instance_norm = True,
        optimizer_kwargs   = params["optimizer_kwargs"],
        add_encoders       = build_encoders(),
        add_relative_index = True,
        pl_trainer_kwargs  = pl_trainer_kwargs,
        random_state       = 42,
        save_checkpoints   = False,
    )

In [ ]:
# ----- datos 2019‑H2 -----
mask_2019 = (df_tft_corto["timestamp"] >= "2019-05-01") & \
            (df_tft_corto["timestamp"] <= "2019-12-31")
df_2019 = df_tft_corto.loc[mask_2019].copy()

# tenemos NaN en covariables
for col in cov_cols_corto:
    if df_2019[col].isnull().any():
        if df_2019[col].dtype in [np.float64, np.float32, np.int64, np.int32]:
            df_2019[col] = df_2019[col].fillna(method="ffill").fillna(0)  # ffill y si arranca con NaN, 0
        else:
            df_2019[col] = df_2019[col].fillna(0)

C:\Users\opera\AppData\Local\Temp\ipykernel_2776\1160341846.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_2019[col] = df_2019[col].fillna(method="ffill").fillna(0)  # ffill y si arranca con NaN, 0


In [133]:
series_19, covs_19 = make_series_and_covs_from_series_id(
    df       = df_2019,
    cov_cols = cov_cols_corto,
    time_col = "timestamp",
    freq     = "MS"
)

📊 262805 series agrupadas
🚀 Procesando grupos...


Creando TimeSeries: 100%|██████████| 262805/262805 [15:06<00:00, 289.83it/s]


✅ 262805 series creadas


In [142]:
# ----- modelo final 6‑1 con best_params -----
tft_corto_final = create_tft_model(best_params, in_len=6, out_len=2)
tft_corto_final.fit(series=series_19, past_covariates=covs_19, verbose=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                              | Type                             | Params | Mode 
------------------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0      | train
1  | val_metrics                       | MetricCollection                 | 0      | train
2  | rin                               | RINorm                           | 2      | train
3  | input_embeddings                  | _MultiEmbedding                  | 0      | train
4  | static_covariates_vsn             | _VariableSelectionNetwork        | 0      | train
5  | encoder_vsn                       | _VariableSelectionNetwork        | 16.2 K | train
6  | decoder_vsn                       | _VariableSelectionNetwork        | 1.6 K  | train
7  | static_cont

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=60` reached.


TFTModel(output_chunk_shift=0, hidden_size=64, lstm_layers=2, num_attention_heads=4, full_attention=False, feed_forward=GatedResidualNetwork, dropout=0.1, hidden_continuous_size=8, categorical_embedding_sizes=None, add_relative_index=True, loss_fn=None, likelihood=QuantileRegression(quantiles=[0.1, 0.5, 0.9], prior_strength=1.0), norm_type=LayerNorm, use_static_covariates=True, input_chunk_length=6, output_chunk_length=2, batch_size=64, n_epochs=60, use_reversible_instance_norm=True, optimizer_kwargs={'lr': 0.001}, add_encoders={'cyclic': {'past': ['month']}, 'transformer': Scaler}, pl_trainer_kwargs={'accelerator': 'gpu', 'devices': 1, 'enable_progress_bar': True}, random_state=42, save_checkpoints=False)

In [143]:
# ----- predicción feb‑2020 (n=2) -----
pred_tft_corto = tft_corto_final.predict(
    n=2,
    series=series_19,
    past_covariates=covs_19,
    verbose=True
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Generating TimeSeries:   0%|          | 0/262805 [00:00<?, ?it/s]

In [144]:
# --- 1. Extraer mediana de febrero 2020 para cada serie (producto_cliente) ---
series_ids = df_tft_corto["series_id"].drop_duplicates().tolist()

preds_feb_corto = []
for i, fcst in enumerate(pred_tft_corto):
    try:
        fcst_med = fcst.quantile_timeseries(0.5)
    except AttributeError:
        fcst_med = fcst

    if "2020-02-01" in fcst_med.time_index:
        idx_feb = fcst_med.time_index.get_loc("2020-02-01")
    elif len(fcst_med) > 1:
        idx_feb = -1
    else:
        idx_feb = 0

    valor_feb_corto = fcst_med.values()[idx_feb, 0]
    series_id = series_ids[i]

    preds_feb_corto.append({
        "series_id": series_id,
        "tn_feb2020": valor_feb_corto
    })

preds_feb_corto = pd.DataFrame(preds_feb_corto)

In [146]:
def check_nans(series_list, covariates_list):
    for i, (ts, cov) in enumerate(zip(series_list, covariates_list)):
        if np.isnan(ts.values()).any():
            print(f"NaN en serie {i} / series_id {series_ids[i]}")
        if np.isnan(cov.values()).any():
            print(f"NaN en covariables {i} / series_id {series_ids[i]}")

check_nans(series_19, covs_19)

In [147]:
targets = np.concatenate([ts.values().flatten() for ts in series_19])
print("Min:", np.min(targets), "Max:", np.max(targets), "Mean:", np.mean(targets))
print("¿Todos ceros?", np.all(targets == 0))
print("¿Todos iguales?", np.all(targets == targets[0]))

Min: 0.0 Max: 547.8785 Mean: 0.115187585
¿Todos ceros? False
¿Todos iguales? False


In [148]:
cov_concat = np.concatenate([cov.values().flatten() for cov in covs_19])
print("NaN en covariables:", np.isnan(cov_concat).sum())
print("Valores máximos/minimos:", np.nanmax(cov_concat), np.nanmin(cov_concat))

NaN en covariables: 0
Valores máximos/minimos: 290.0 0.0


In [ ]:
targets = np.concatenate([ts.values().flatten() for ts in series_19])
print("Min:", np.min(targets), "Max:", np.max(targets), "Mean:", np.mean(targets))
print("¿Todos ceros?", np.all(targets == 0))
print("¿Todos iguales?", np.all(targets == targets[0]))

In [145]:
# --- 2. Mapeo directo a product_id ---
preds_feb_corto["product_id"] = preds_feb_corto["series_id"].map(series_to_product)

preds_feb_corto["tn_feb2020_clipped"] = preds_feb_corto["tn_feb2020"].clip(lower=0)

# --- 3. Sumar todos los clientes (series) de cada producto ---
resultado_tft_corto = (
    preds_feb_corto
      .groupby("product_id", as_index=False)["tn_feb2020_clipped"]
      .sum()
      .rename(columns={"tn_feb2020_clipped": "pred_tft_corto"})
)

print(resultado_tft_corto.head())

   product_id  pred_tft_corto
0       20001       27.722528
1       20002       58.231617
2       20003       10.315485
3       20004        9.664122
4       20005       19.407740


### Al igual que el TFT largo, las predicciones quedan muy debajo del baseline

In [151]:
resultado_tft_corto.head()

,product_id,pred_tft_corto
0,20001,27.722528
1,20002,58.231617
2,20003,10.315485
3,20004,9.664122
4,20005,19.407740


### TFT largo

In [71]:
params_default = {
    "hidden_size":        64,
    "n_heads":            4,
    "dropout":            0.1,
    "batch_size":         64,
    "optimizer_kwargs":   {"lr": 1e-3},
}

model_largo = create_tft_model(params_default, in_len=34, out_len=2)

In [72]:
model_largo.fit(
    series           = tft_largo["series_list"],
    past_covariates  = tft_largo["covariates_list"],
    verbose          = True
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                              | Type                             | Params | Mode 
------------------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0      | train
1  | val_metrics                       | MetricCollection                 | 0      | train
2  | rin                               | RINorm                           | 2      | train
3  | input_embeddings                  | _MultiEmbedding                  | 0      | train
4  | static_covariates_vsn             | _VariableSelectionNetwork        | 0      | train
5  | encoder_vsn                       | _VariableSelectionNetwork        | 16.2 K | train
6  | decoder_vsn                       | _VariableSelectionNetwork        | 1.6 K  | train
7  | static_cont

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=80` reached.


TFTModel(output_chunk_shift=0, hidden_size=64, lstm_layers=2, num_attention_heads=4, full_attention=False, feed_forward=GatedResidualNetwork, dropout=0.1, hidden_continuous_size=8, categorical_embedding_sizes=None, add_relative_index=True, loss_fn=None, likelihood=QuantileRegression(quantiles=[0.1, 0.5, 0.9], prior_strength=1.0), norm_type=LayerNorm, use_static_covariates=True, input_chunk_length=34, output_chunk_length=2, batch_size=64, n_epochs=80, use_reversible_instance_norm=True, optimizer_kwargs={'lr': 0.001}, add_encoders={'cyclic': {'past': ['month']}, 'transformer': Scaler}, pl_trainer_kwargs={'accelerator': 'gpu', 'devices': 1, 'enable_progress_bar': True}, random_state=42, save_checkpoints=False)

In [75]:
pred_model_largo = model_largo.predict(
    n               = 2,
    series          = tft_largo["series_list"],
    past_covariates = tft_largo["covariates_list"],
    verbose=True
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Generating TimeSeries:   0%|          | 0/262805 [00:00<?, ?it/s]

In [107]:
# acá genero df
# --- 1. Extraer mediana de febrero 2020 para cada serie (producto_cliente) ---
series_ids = df_tft_largo["series_id"].drop_duplicates().tolist()

preds_feb_largo = []
for i, fcst in enumerate(pred_model_largo):
    try:
        fcst_med = fcst.quantile_timeseries(0.5)
    except AttributeError:
        fcst_med = fcst

    if "2020-02-01" in fcst_med.time_index:
        idx_feb = fcst_med.time_index.get_loc("2020-02-01")
    elif len(fcst_med) > 1:
        idx_feb = -1
    else:
        idx_feb = 0

    valor_feb_largo = fcst_med.values()[idx_feb, 0]
    series_id = series_ids[i]

    preds_feb_largo.append({
        "series_id": series_id,
        "tn_feb2020": valor_feb_largo
    })

preds_feb_largo = pd.DataFrame(preds_feb_largo)

# --- 2. Mapeo directo a product_id ---
preds_feb_largo["product_id"] = preds_feb_largo["series_id"].map(series_to_product)

preds_feb_largo["tn_feb2020_clipped"] = preds_feb_largo["tn_feb2020"].clip(lower=0)

# --- 3. Sumar todos los clientes (series) de cada producto ---
resultado_tft_largo = (
    preds_feb_largo
      .groupby("product_id", as_index=False)["tn_feb2020_clipped"]
      .sum()
      .rename(columns={"tn_feb2020_clipped": "pred_tft_largo"})
)

print(resultado_tft_largo.head())

   product_id  pred_tft_largo
0       20001       69.889557
1       20002        9.226202
2       20003      508.837769
3       20004       23.788918
4       20005       21.414722


In [108]:
print("Sumatoria de negativos antes del clip:",
      preds_feb_largo[preds_feb_largo["tn_feb2020"] < 0]["tn_feb2020"].sum())
print("Cantidad de clientes con predicción negativa:",
      (preds_feb_largo["tn_feb2020"] < 0).sum())

Sumatoria de negativos antes del clip: -332.09457
Cantidad de clientes con predicción negativa: 107889


In [102]:
# ---------- AUDITORÍA DE COVERAGE EN PREDICCIÓN ----------

print("== Coverage y chequeo de granularidad en predicciones ==")

# 1. ¿Cuántos clientes únicos hay en la predicción de febrero 2020 para cada producto?
clientes_predichos = preds_feb_largo.groupby("product_id")["series_id"].nunique()
print("\nClientes únicos predichos (feb 2020) por producto:\n", clientes_predichos.head(10))

# 2. ¿Cuántos clientes únicos hubo históricamente para cada producto?
clientes_historicos = df_tft_largo.groupby("product_id")["customer_id"].nunique()
print("\nClientes históricos por producto:\n", clientes_historicos.head(10))

# 3. Merge para ver si falta coverage en la predicción
coverage = pd.DataFrame({
    "clientes_predichos": clientes_predichos,
    "clientes_historicos": clientes_historicos
}).fillna(0).astype(int)
coverage["coverage_%"] = 100 * coverage["clientes_predichos"] / coverage["clientes_historicos"].replace(0, 1)
print("\nCoverage (% clientes predichos / históricos):\n", coverage.head(10))
print("\nCoverage summary:\n", coverage["coverage_%"].describe())

# 4. ¿Hay muchos NaN en las predicciones?
n_nan = preds_feb_largo["tn_feb2020"].isnull().sum()
print(f"\nCantidad de NaN en predicciones: {n_nan} de {len(preds_feb_largo)} filas")

# 5. ¿Cómo se ve la suma agregada por producto vs histórico?
total_predicho = resultado_tft_largo["pred_tft_largo"].sum()
print(f"\nTotal predicho (sumado en todos los productos): {total_predicho:,.2f}")

# (Opcional) ¿Cómo era el histórico del último año real disponible?
if "timestamp" in df_tft_largo.columns:
    # Busca el mes real más cercano a febrero
    meses_disponibles = df_tft_largo["timestamp"].sort_values().unique()
    if "2019-02-01" in meses_disponibles:
        mes_compara = "2019-02-01"
    elif "2019-12-01" in meses_disponibles:
        mes_compara = "2019-12-01"
    else:
        mes_compara = meses_disponibles[-1]  # último mes disponible
    
    total_real = df_tft_largo[df_tft_largo["timestamp"] == mes_compara].groupby("product_id")["tn"].sum().sum()
    print(f"Total histórico para el mes {mes_compara}: {total_real:,.2f}")
else:
    print("No se encontró la columna timestamp para comparar histórico.")

# 6. Muestra algunos casos donde coverage es bajo
coverage_bajo = coverage[coverage["coverage_%"] < 80]
if not coverage_bajo.empty:
    print("\nProductos con coverage bajo (<80%):")
    print(coverage_bajo.head(10))

print("\n== FIN DE AUDITORÍA ==")

== Coverage y chequeo de granularidad en predicciones ==

Clientes únicos predichos (feb 2020) por producto:
 product_id
20001    444
20002    445
20003    402
20004    415
20005    378
20006    470
20007    482
20008    473
20009    424
20010    423
Name: series_id, dtype: int64

Clientes históricos por producto:
 product_id
20001    444
20002    445
20003    402
20004    415
20005    378
20006    470
20007    482
20008    473
20009    424
20010    423
Name: customer_id, dtype: int64

Coverage (% clientes predichos / históricos):
             clientes_predichos  clientes_historicos  coverage_%
product_id                                                     
20001                      444                  444       100.0
20002                      445                  445       100.0
20003                      402                  402       100.0
20004                      415                  415       100.0
20005                      378                  378       100.0
20006         

### No hay faltantes, la misma cantidad de predicciones (clientes) por producto pero la diferencia es muy grande solo mirando los primeros clientes (68tn para el primer cliente es muy bajo)
### Una serie tan grande donde se hizo el producto cartesiano para rellenar hace que las predicciones del modelo sean muy bajas, habría que considerar hacer un modelo solo con top n de clientes

In [111]:
# pruebo mirando otro quantil
preds_feb_largo_09 = []
for i, fcst in enumerate(pred_model_largo):
    try:
        fcst_med = fcst.quantile_timeseries(0.9)
    except AttributeError:
        fcst_med = fcst

    if "2020-02-01" in fcst_med.time_index:
        idx_feb = fcst_med.time_index.get_loc("2020-02-01")
    elif len(fcst_med) > 1:
        idx_feb = -1
    else:
        idx_feb = 0

    valor_feb_largo = fcst_med.values()[idx_feb, 0]
    series_id = series_ids[i]
    product_id = series_to_product.get(series_id, None)
    preds_feb_largo_09.append({
        "series_id": series_id,
        "product_id": product_id,
        "tn_feb2020": valor_feb_largo
    })

preds_feb_largo_09 = pd.DataFrame(preds_feb_largo_09)
preds_feb_largo_09["tn_feb2020_clipped"] = preds_feb_largo_09["tn_feb2020"].clip(lower=0)
resultado_tft_largo_09 = (
    preds_feb_largo_09
      .groupby("product_id", as_index=False)["tn_feb2020_clipped"]
      .sum()
      .rename(columns={"tn_feb2020_clipped": "pred_tft_largo_09"})
)
print(resultado_tft_largo_09.head())

   product_id  pred_tft_largo_09
0       20001          69.889557
1       20002           9.226202
2       20003         508.837769
3       20004          23.788918
4       20005          21.414722


#### No hay mayor diferencia

In [110]:
# Sumar predicción por cliente para un producto clave
prod_key = 20001
print(preds_feb_largo[preds_feb_largo["product_id"] == prod_key].sort_values("tn_feb2020", ascending=False).head(10))

         series_id  tn_feb2020  product_id  tn_feb2020_clipped
31181  10351_20001   10.567575       20001           10.567575
31236  10570_20001    8.841102       20001            8.841102
31187  10018_20001    4.366196       20001            4.366196
31222  10062_20001    3.673208       20001            3.673208
31212  10137_20001    3.635326       20001            3.635326
31214  10050_20001    3.570915       20001            3.570915
31204  10272_20001    3.166698       20001            3.166698
31184  10144_20001    2.765589       20001            2.765589
31198  10225_20001    2.759619       20001            2.759619
31197  10082_20001    1.875728       20001            1.875728


In [105]:
mes_compara = "2019-02-01"
df_2019 = df_tft_largo[(df_tft_largo["timestamp"] == mes_compara) & (df_tft_largo["product_id"] == prod_key)]
print(df_2019[["customer_id", "tn"]].sort_values("tn", ascending=False).head(10))

         customer_id          tn
1118329        10001  309.906097
1121893        10004  155.261108
1123981        10013  124.948219
1118617        10007  108.066658
1124629        10003   81.215218
1122109        10015   80.587914
1122757        10018   37.202171
1118833        10008   31.421890
2541013        10034   25.764830
2542921        10022   22.180161


In [ ]:
cliente_key = 10001  # uno de los más grandes
print(df_tft_largo[(df_tft_largo["product_id"] == prod_key) & (df_tft_largo["customer_id"] == cliente_key)].sort_values("timestamp")[["timestamp", "tn"]])

### TFT Agg

In [160]:
df_tft_agg["series_id"] = df_tft_agg["product_id"].astype(str)

In [162]:
grouped = df_tft_agg.groupby("series_id")
print(f"📊 {len(grouped)} series agrupadas")

series_list, covariates_list = [], []
print("🚀 Procesando grupos...")
for series_id, group in tqdm(grouped, desc="Creando TimeSeries"):
    ts = TimeSeries.from_dataframe(
        group,
        time_col='timestamp',
        value_cols='tn',
        freq='MS'
    )

    series_list.append(ts)

print(f"✅ {len(series_list)} series creadas")

📊 780 series agrupadas
🚀 Procesando grupos...


Creando TimeSeries: 100%|██████████| 780/780 [00:01<00:00, 516.82it/s]

✅ 780 series creadas


In [ ]:
print("series dtype:", series_list_agg[0].dtype)

📊 780 grupos creados
🚀 Procesando grupos...


Creando TimeSeries: 100%|██████████| 780/780 [00:03<00:00, 251.19it/s]


In [163]:
def create_tft_model_nofuture(
    input_chunk_length: int,
    output_chunk_length: int,
    max_epochs: int = 200,
    batch_size: int = 256,
    hidden_size: int = 512,
    lstm_layers: int = 3,
    num_attention_heads: int = 4,
    dropout: float = 0.1,
    quantiles = [0.1, 0.5, 0.9],
    lr: float = 1e-4,
    random_state: int = 42,
):
    return TFTModel(
        input_chunk_length           = input_chunk_length,
        output_chunk_length          = output_chunk_length,
        hidden_size                  = hidden_size,
        lstm_layers                  = lstm_layers,
        num_attention_heads          = num_attention_heads,
        dropout                      = dropout,
        batch_size                   = batch_size,
        n_epochs                     = max_epochs,
        likelihood                   = QuantileRegression(quantiles),
        use_reversible_instance_norm = True,        # escalado interno del target
        optimizer_kwargs             = {"lr": lr},
        random_state                 = random_state,
        add_encoders                 = {
            # estacionalidad sola; Darts generará past y future con month
            "cyclic": {"past": ["month"], "future": ["month"]},
        },
        pl_trainer_kwargs            = dict(
            accelerator="gpu",
            devices=1,
            enable_progress_bar=True,
        ),
        save_checkpoints             = True,
        force_reset                  = True,
    )

In [164]:
tft_agg['model'] = create_tft_model_nofuture(
    input_chunk_length  = 34,
    output_chunk_length = 2,
    max_epochs          = 250,
    batch_size          = 256,
)

In [165]:
#entrenamos sin variables macro

tft_agg['model'].fit(
    series  = series_list,
    verbose = True
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                              | Type                             | Params | Mode 
------------------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0      | train
1  | val_metrics                       | MetricCollection                 | 0      | train
2  | rin                               | RINorm                           | 2      | train
3  | input_embeddings                  | _MultiEmbedding                  | 0      | train
4  | static_covariates_vsn             | _VariableSelectionNetwork        | 0      | train
5  | encoder_vsn                       | _VariableSelectionNetwork        | 62.6 K | train
6  | decoder_vsn                       | _VariableSelectionNetwork        | 25.0 K | train
7  | static_cont

Training: |          | 0/? [00:00<?, ?it/s]

c:\Users\opera\AppData\Local\Programs\Python\Python313\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:384: `ModelCheckpoint(monitor='val_loss')` could not find the monitored key in the returned metrics: ['train_loss', 'epoch', 'step']. HINT: Did you call `log('val_loss', value)` in the `LightningModule`?
`Trainer.fit` stopped: `max_epochs=250` reached.


TFTModel(output_chunk_shift=0, hidden_size=512, lstm_layers=3, num_attention_heads=4, full_attention=False, feed_forward=GatedResidualNetwork, dropout=0.1, hidden_continuous_size=8, categorical_embedding_sizes=None, add_relative_index=False, loss_fn=None, likelihood=QuantileRegression(quantiles=[0.1, 0.5, 0.9], prior_strength=1.0), norm_type=LayerNorm, use_static_covariates=True, input_chunk_length=34, output_chunk_length=2, batch_size=256, n_epochs=250, use_reversible_instance_norm=True, optimizer_kwargs={'lr': 0.0001}, random_state=42, add_encoders={'cyclic': {'past': ['month'], 'future': ['month']}}, pl_trainer_kwargs={'accelerator': 'gpu', 'devices': 1, 'enable_progress_bar': True}, save_checkpoints=True, force_reset=True)

In [167]:
predictions_agg = tft_agg['model'].predict(
    n       = 2,
    series  = series_list
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [168]:
preds_med_agg = []
for fcst in predictions_agg:
    try:
        fcst_agg = fcst.quantile_timeseries(0.5)
    except AttributeError:
        fcst_agg = fcst
    preds_med_agg.append(fcst_agg)

# mapping series->product
series_ids = (
    df_tft_agg.drop_duplicates("series_id")["series_id"].tolist()
)

In [169]:
rows = []
for sid, fcst_agg, ts in zip(series_ids, preds_med_agg, series_list):
    vals = fcst_agg.values().flatten()
    # Por sanity, los últimos 2 observados reales
    y_last = ts[-2:].values().flatten()
    
    # Clip a 0 por si la predicción es negativa
    pred_jan = max(vals[0], 0) if len(vals) > 0 else np.nan
    pred_feb = max(vals[1], 0) if len(vals) > 1 else np.nan

    rows.append({
        "product_id": sid,                   # sid es el product_id directo
        "pred_jan2020": pred_jan,
        "pred_feb2020": pred_feb,
        "last_obs_tminus1": y_last[0] if len(y_last) > 0 else np.nan,
        "last_obs_tminus0": y_last[1] if len(y_last) > 1 else np.nan,
    })

df_pred_agg = pd.DataFrame(rows)
print(df_pred_agg.head())

  product_id  pred_jan2020  pred_feb2020  last_obs_tminus1  last_obs_tminus0
0      20001   1580.775757   1328.167847       1397.372314       1504.688599
1      20002    868.006592    849.866272       1423.577393       1087.308594
2      20003    661.217590    660.033691        948.293945        892.501282
3      20004    561.623230    497.657654        723.942078        637.900024
4      20005    514.831543    425.417480        606.911743        593.244446


In [196]:
pred_agg = df_pred_agg[["product_id", "pred_feb2020"]].copy()
pred_agg = pred_agg.rename(columns={"pred_feb2020": "pred_tft_agg"})

pred_agg.head()

,product_id,pred_tft_agg
0,20001,1328.167847
1,20002,849.866272
2,20003,660.033691
3,20004,497.657654
4,20005,425.417480


### Resultados finales

In [270]:
comparativa = baseline.rename(columns={"tn": "reg_lin"}).copy()
comparativa.head()

,product_id,reg_lin
0,20001,1162.705930
1,20002,1183.639540
2,20003,684.763159
3,20004,580.484232
4,20005,563.559924


In [271]:
comparativa = comparativa.merge(resultado_tft_largo, on="product_id", how="left") 
comparativa = comparativa.merge(resultado_tft_corto, on="product_id", how="left") 
comparativa = comparativa.merge(pred_agg, on="product_id", how="left") 

In [272]:
comparativa = comparativa.rename(columns={
    "pred_tft_largo": "tft_largo",
    "pred_tft_corto": "tft_corto",
    "pred_tft_agg":   "tft_agg"
})

In [273]:
comparativa.head()

,product_id,reg_lin,tft_largo,tft_corto,tft_agg
0,20001,1162.705930,69.889557,27.722528,1328.167847
1,20002,1183.639540,9.226202,58.231617,849.866272
2,20003,684.763159,508.837769,10.315485,660.033691
3,20004,580.484232,23.788918,9.664122,497.657654
4,20005,563.559924,21.414722,19.407740,425.417480


In [274]:
# segmento el product_id por volumen

# volumen total por producto ---
vol_por_producto = df_tft_agg.groupby("product_id")["tn"].sum().reset_index(name="vol_total")

# cuantiles para separar
vol_por_producto["q_volumen"] = pd.qcut(vol_por_producto["vol_total"], q=4, labels=[4, 3, 2, 1])

#  Merge con tu comparativa ---
comparativa = comparativa.merge(vol_por_producto[["product_id", "q_volumen"]], on="product_id", how="left")

In [275]:
comparativa.head()

,product_id,reg_lin,tft_largo,tft_corto,tft_agg,q_volumen
0,20001,1162.705930,69.889557,27.722528,1328.167847,1
1,20002,1183.639540,9.226202,58.231617,849.866272,1
2,20003,684.763159,508.837769,10.315485,660.033691,1
3,20004,580.484232,23.788918,9.664122,497.657654,1
4,20005,563.559924,21.414722,19.407740,425.417480,1


In [276]:
print("\n=== Error absoluto promedio por cuantil de volumen ===")
print(
    comparativa.assign(
        err_tft_largo = (comparativa["tft_largo"] - comparativa["reg_lin"]).abs(),
        err_tft_corto = (comparativa["tft_corto"] - comparativa["reg_lin"]).abs(),
        err_tft_agg   = (comparativa["tft_agg"]   - comparativa["reg_lin"]).abs(),
    ).groupby("q_volumen", observed=True)[["err_tft_largo", "err_tft_corto", "err_tft_agg"]].mean()
)

print("\n=== Error relativo promedio por cuantil de volumen ===")
print(
    comparativa.assign(
        relerr_tft_largo = (comparativa["tft_largo"] - comparativa["reg_lin"]).abs() / (comparativa["reg_lin"].abs() + 1e-8),
        relerr_tft_corto = (comparativa["tft_corto"] - comparativa["reg_lin"]).abs() / (comparativa["reg_lin"].abs() + 1e-8),
        relerr_tft_agg   = (comparativa["tft_agg"]   - comparativa["reg_lin"]).abs() / (comparativa["reg_lin"].abs() + 1e-8),
    ).groupby("q_volumen", observed=True)[["relerr_tft_largo", "relerr_tft_corto", "relerr_tft_agg"]].mean()
)

print("\n=== Error absoluto TOTAL por cuantil de volumen ===")
print(
    comparativa.assign(
        err_tft_largo = (comparativa["tft_largo"] - comparativa["reg_lin"]).abs(),
        err_tft_corto = (comparativa["tft_corto"] - comparativa["reg_lin"]).abs(),
        err_tft_agg   = (comparativa["tft_agg"]   - comparativa["reg_lin"]).abs(),
    ).groupby("q_volumen", observed=True)[["err_tft_largo", "err_tft_corto", "err_tft_agg"]].sum()
)


=== Error absoluto promedio por cuantil de volumen ===
           err_tft_largo  err_tft_corto  err_tft_agg
q_volumen                                           
4              11.301856      14.928827     0.732892
3              25.334954      22.054953     2.137368
2              41.825602      29.432640     5.624793
1             109.321647      96.301473    25.127104

=== Error relativo promedio por cuantil de volumen ===
           relerr_tft_largo  relerr_tft_corto  relerr_tft_agg
q_volumen                                                    
4                  9.804962         14.514217        0.488385
3                  7.754419          5.700748        0.365678
2                  4.002938          2.398241        0.347807
1                  1.076002          0.743463        0.251129

=== Error absoluto TOTAL por cuantil de volumen ===
           err_tft_largo  err_tft_corto  err_tft_agg
q_volumen                                           
4            2203.861927    2911.121349

### Si bien el error relativo es alto en los q4 (poco volumen), en la métrica final los de mayor impacto son los q1.
### Si utilizamos la regresión como  baseline, el tft_agg es el de performance más similar

In [277]:
# intento crear un "febrero 2020" mirando años anteriores y comportamiento como ultimo trimestre

df_febproxy = df_tft_agg.copy()

df_febproxy["timestamp"] = pd.to_datetime(df_febproxy["timestamp"])

# año y mes para facilitar filtros
df_febproxy["year"] = df_febproxy["timestamp"].dt.year
df_febproxy["month"] = df_febproxy["timestamp"].dt.month

# Diccionario para guardar el ratio de cada producto y año
ratios = []

for year in [2018, 2019]:
    # Octubre-diciembre del año anterior
    oct_nov_dic = df_febproxy.query("year == @year-1 and month in [10, 11, 12]") \
        .groupby("product_id")["tn"].mean().rename("mean_q4")
    # Febrero del año actual
    feb = df_febproxy.query("year == @year and month == 2") \
        .set_index("product_id")["tn"]
    # ratio solo donde hay ambos valores
    df_merge = pd.concat([oct_nov_dic, feb], axis=1, join="inner").dropna()
    df_merge["ratio"] = df_merge["tn"] / (df_merge["mean_q4"] + 1e-8)
    df_merge["year"] = year
    ratios.append(df_merge[["ratio"]])

# ratios históricos
ratios_df = pd.concat(ratios, keys=[2018, 2019])
ratios_df = ratios_df.reset_index().rename(columns={"level_0": "year"})

# ratio promedio histórico por producto
ratio_mean = ratios_df.groupby("product_id")["ratio"].mean().rename("ratio_mean")

# promedio de oct-nov-dic 2019 para cada producto
oct_nov_dic_2019 = df_febproxy.query("year == 2019 and month in [10, 11, 12]") \
    .groupby("product_id")["tn"].mean().rename("mean_q4_2019")

# proxy de febrero 2020
proxy_feb2020 = (oct_nov_dic_2019 * ratio_mean).rename("proxy_feb2020").reset_index()
mediana = proxy_feb2020.loc[proxy_feb2020["proxy_feb2020"] < 1e4, "proxy_feb2020"].median()

proxy_feb2020.loc[proxy_feb2020["proxy_feb2020"] > 1e4, "proxy_feb2020"] = mediana

proxy_feb2020.head()

,product_id,proxy_feb2020
0,20001,1074.649048
1,20002,1221.913574
2,20003,601.752380
3,20004,467.332245
4,20005,480.747131


In [278]:
comparativa = comparativa.merge(proxy_feb2020, on="product_id", how="left")
comparativa.head()

,product_id,reg_lin,tft_largo,tft_corto,tft_agg,q_volumen,proxy_feb2020
0,20001,1162.705930,69.889557,27.722528,1328.167847,1,1074.649048
1,20002,1183.639540,9.226202,58.231617,849.866272,1,1221.913574
2,20003,684.763159,508.837769,10.315485,660.033691,1,601.752380
3,20004,580.484232,23.788918,9.664122,497.657654,1,467.332245
4,20005,563.559924,21.414722,19.407740,425.417480,1,480.747131


In [279]:
oct_val = df_febproxy.query("year == 2019 and month == 10").set_index("product_id")["tn"]
nov_val = df_febproxy.query("year == 2019 and month == 11").set_index("product_id")["tn"]
dec_val = df_febproxy.query("year == 2019 and month == 12").set_index("product_id")["tn"]

# Delta entre diciembre y octubre 
delta_oct_dec = (dec_val - oct_val).rename("delta_oct_dec")

# Delta promedio mes a mes
delta_mean = ((nov_val - oct_val) + (dec_val - nov_val)) / 2
delta_mean = delta_mean.rename("delta_mean_q4")

delta_df = pd.concat([delta_oct_dec, delta_mean], axis=1).reset_index()
delta_df.head()

,product_id,delta_oct_dec,delta_mean_q4
0,20001,-56.816895,-28.408447
1,20002,-892.227783,-446.113892
2,20003,-188.865173,-94.432587
3,20004,-426.796265,-213.398132
4,20005,-403.538330,-201.769165


In [280]:
comparativa = comparativa.merge(delta_df, on="product_id", how="left")
comparativa.head()

,product_id,reg_lin,tft_largo,tft_corto,tft_agg,q_volumen,proxy_feb2020,delta_oct_dec,delta_mean_q4
0,20001,1162.705930,69.889557,27.722528,1328.167847,1,1074.649048,-56.816895,-28.408447
1,20002,1183.639540,9.226202,58.231617,849.866272,1,1221.913574,-892.227783,-446.113892
2,20003,684.763159,508.837769,10.315485,660.033691,1,601.752380,-188.865173,-94.432587
3,20004,580.484232,23.788918,9.664122,497.657654,1,467.332245,-426.796265,-213.398132
4,20005,563.559924,21.414722,19.407740,425.417480,1,480.747131,-403.538330,-201.769165


In [281]:
subir = pd.DataFrame({
    "product_id": comparativa["product_id"],
    "reg_lin": comparativa["reg_lin"],
    "tft_agg": comparativa["tft_agg"],
    "blend_05": np.where(
        comparativa["q_volumen"] == 1,
        0.5 * comparativa["reg_lin"] + 0.5 * comparativa["tft_agg"],
        comparativa["reg_lin"]
    ),
    "blend_06": np.where(
        comparativa["q_volumen"] == 1,
        0.6 * comparativa["reg_lin"] + 0.4 * comparativa["tft_agg"],
        comparativa["reg_lin"]
    ),
    "blend_07": np.where(
        comparativa["q_volumen"] == 1,
        0.7 * comparativa["reg_lin"] + 0.3 * comparativa["tft_agg"],
        comparativa["reg_lin"]
    ),
    "blend_delta": [
        (0.3 * rl + 0.7 * ta) if (q == 1 and delta > 0) else
        (0.7 * rl + 0.3 * ta) if (q == 1 and delta <= 0) else
        ta
        for rl, ta, delta, q in zip(
            comparativa["reg_lin"],
            comparativa["tft_agg"],
            comparativa["delta_mean_q4"],
            comparativa["q_volumen"]
        )
    ]
})

subir.head()

,product_id,reg_lin,tft_agg,blend_05,blend_06,blend_07,blend_delta
0,20001,1162.705930,1328.167847,1245.436888,1228.890697,1212.344505,1212.344505
1,20002,1183.639540,849.866272,1016.752906,1050.130233,1083.507560,1083.507560
2,20003,684.763159,660.033691,672.398425,674.871372,677.344319,677.344319
3,20004,580.484232,497.657654,539.070943,547.353601,555.636259,555.636259
4,20005,563.559924,425.417480,494.488702,508.302946,522.117191,522.117191


In [282]:
subir_proxy = subir.merge(comparativa[["product_id", "proxy_feb2020"]], on="product_id", how="left")

for col in ["reg_lin", "tft_agg", "blend_05", "blend_06", "blend_07", "blend_delta"]:
    mae = (subir_proxy[col] - subir_proxy["proxy_feb2020"]).abs().mean()
    print(f"MAE vs proxy para {col}: {mae:.2f}")

MAE vs proxy para reg_lin: 9.78
MAE vs proxy para tft_agg: 12.07
MAE vs proxy para blend_05: 10.25
MAE vs proxy para blend_06: 10.05
MAE vs proxy para blend_07: 9.89
MAE vs proxy para blend_delta: 10.20


In [266]:
# 1. Mirá algunos valores del proxy y de los blends
print(subir_proxy[["product_id", "proxy_feb2020", "reg_lin", "tft_agg", "blend_05"]].head(10))

# 2. Revisá si hay NaNs
print("Proxy tiene NaNs:", subir_proxy["proxy_feb2020"].isna().sum())
print("reg_lin tiene NaNs:", subir_proxy["reg_lin"].isna().sum())

   product_id  proxy_feb2020      reg_lin      tft_agg     blend_05
0       20001    1074.649048  1162.705930  1328.167847  1245.436888
1       20002    1221.913574  1183.639540   849.866272  1016.752906
2       20003     601.752380   684.763159   660.033691   672.398425
3       20004     467.332245   580.484232   497.657654   539.070943
4       20005     480.747131   563.559924   425.417480   494.488702
5       20006     403.603058   482.886451   452.452118   467.669285
6       20007     262.328125   390.923780   427.053833   408.988806
7       20008     318.246948   422.339743   399.852478   411.096111
8       20009     565.203369   450.559917   546.069641   498.314779
9       20010     288.431702   418.689170   199.366974   309.028072
Proxy tiene NaNs: 0
reg_lin tiene NaNs: 0


In [284]:
# optimizacion por mae contra proxy

best_mae = np.inf
best_alpha = None
best_beta = None

for alpha in np.arange(0.1, 1.0, 0.1):  # Para delta > 0
    for beta in np.arange(0.1, 1.0, 0.1):  # Para delta <= 0
        pred = [
            alpha * rl + (1 - alpha) * ta if delta > 0
            else beta * rl + (1 - beta) * ta
            for rl, ta, delta in zip(comparativa["reg_lin"], comparativa["tft_agg"], comparativa["delta_mean_q4"])
        ]
        mae = np.mean(np.abs(np.array(pred) - comparativa["proxy_feb2020"]))
        if mae < best_mae:
            best_mae = mae
            best_alpha = alpha
            best_beta = beta

print(f"Mejor MAE: {best_mae:.2f} con alpha={best_alpha:.2f}, beta={best_beta:.2f}")

Mejor MAE: 9.69 con alpha=0.90, beta=0.80


In [287]:
subir.head()

,product_id,reg_lin,tft_agg,blend_05,blend_06,blend_07,blend_delta
0,20001,1162.705930,1328.167847,1245.436888,1228.890697,1212.344505,1212.344505
1,20002,1183.639540,849.866272,1016.752906,1050.130233,1083.507560,1083.507560
2,20003,684.763159,660.033691,672.398425,674.871372,677.344319,677.344319
3,20004,580.484232,497.657654,539.070943,547.353601,555.636259,555.636259
4,20005,563.559924,425.417480,494.488702,508.302946,522.117191,522.117191


In [288]:
mask = comparativa["q_volumen"] == 1
cols = ["reg_lin", "tft_agg", "blend_05", "blend_06", "blend_07", "blend_delta"]
for col in cols:
    mae = (subir.loc[mask, col] - comparativa.loc[mask, "proxy_feb2020"]).abs().mean()
    print(f"MAE en q1 para {col}: {mae:.2f}")

MAE en q1 para reg_lin: 24.92
MAE en q1 para tft_agg: 33.77
MAE en q1 para blend_05: 26.80
MAE en q1 para blend_06: 25.99
MAE en q1 para blend_07: 25.37
MAE en q1 para blend_delta: 26.29
